# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
weather_data = pd.read_csv("../WeatherPy/weather_data.csv")
weather_data


,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,‘Āhuimanu,21.4447,-157.8378,83.91,63,40,16.11,US,1634946776
1,1,Ginir,7.1333,40.7000,61.23,44,67,3.40,ET,1634946976
2,2,Ushuaia,-54.8000,-68.3000,46.06,66,75,35.68,AR,1634946777
3,3,Lorengau,-2.0226,147.2712,85.30,66,74,7.92,PG,1634946976
4,4,Nikolskoye,59.7035,30.7861,44.87,72,100,13.20,RU,1634946977
...,...,...,...,...,...,...,...,...,...,...
545,545,Mairi,-11.7114,-40.1489,73.67,81,97,10.07,BR,1634947101
546,546,Wiang Sa,8.6359,99.3666,80.67,91,99,1.16,TH,1634947101
547,547,Lamu,-2.2717,40.9020,76.41,81,28,8.75,KE,1634947101
548,548,Deoli,25.7500,75.3833,68.38,66,0,6.87,IN,1634947102


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
#configure gmaps
gmaps.configure(api_key=gkey)

#Use the Lat and Lng as locations
locations = weather_data[["Latitude","Longitude"]].astype(float)


In [25]:
#Add Heatmap layer to map
humidity = weather_data["Humidity"]  
#Plot heatmap 
fig = gmaps.figure(center=(10.0, 40.0), zoom_level=2)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
ideal_weather = weather_data.loc[(weather_data["Max Temperature"]>= 80) & (weather_data["Max Temperature"]<= 95) 
                                 & (weather_data["Wind Speed"]<= 10) & (weather_data["Cloudiness"]>= 10)].dropna()
ideal_weather
# check length for api calls
# len(ideal_weather)

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Lorengau,-2.0226,147.2712,85.30,66,74,7.92,PG,1634946976
12,12,Tagusao,9.1924,117.8141,81.86,80,100,3.09,PH,1634946978
60,60,Monteagudo,-19.8167,-63.9833,89.38,56,47,1.23,BO,1634946989
70,70,Dingle,10.9995,122.6711,80.24,85,81,9.93,PH,1634946990
78,78,Makkah al Mukarramah,21.4267,39.8261,82.00,40,24,2.10,SA,1634946742
95,95,Kupang,-10.1667,123.5833,84.78,74,75,8.05,ID,1634946996
108,108,Kapaa,22.0752,-159.3190,82.38,77,19,5.01,US,1634946998
110,110,Coahuayana Viejo,18.7333,-103.6833,87.28,86,92,4.25,MX,1634946952
112,112,Muncar,-8.4333,114.3333,80.51,78,100,3.94,ID,1634946998
119,119,Kavaratti,10.5669,72.6420,81.23,77,58,4.03,IN,1634946743


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
# create hotel_df dataframe
hotel_df = pd.DataFrame(ideal_weather)
hotel_df
# add hotel name 
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,Lorengau,-2.0226,147.2712,85.30,66,74,7.92,PG,1634946976,
12,12,Tagusao,9.1924,117.8141,81.86,80,100,3.09,PH,1634946978,
60,60,Monteagudo,-19.8167,-63.9833,89.38,56,47,1.23,BO,1634946989,
70,70,Dingle,10.9995,122.6711,80.24,85,81,9.93,PH,1634946990,
78,78,Makkah al Mukarramah,21.4267,39.8261,82.00,40,24,2.10,SA,1634946742,
95,95,Kupang,-10.1667,123.5833,84.78,74,75,8.05,ID,1634946996,
108,108,Kapaa,22.0752,-159.3190,82.38,77,19,5.01,US,1634946998,
110,110,Coahuayana Viejo,18.7333,-103.6833,87.28,86,92,4.25,MX,1634946952,
112,112,Muncar,-8.4333,114.3333,80.51,78,100,3.94,ID,1634946998,
119,119,Kavaratti,10.5669,72.6420,81.23,77,58,4.03,IN,1634946743,


In [46]:
 #Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : gkey}
# get info from api
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("n/a")

n/a
n/a
n/a
n/a
n/a
n/a
n/a


In [49]:
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,Lorengau,-2.0226,147.2712,85.30,66,74,7.92,PG,1634946976,Seeadler Bay Hotel
12,12,Tagusao,9.1924,117.8141,81.86,80,100,3.09,PH,1634946978,"Shaman's Breath- Beach, Bed & Breakfast"
60,60,Monteagudo,-19.8167,-63.9833,89.38,56,47,1.23,BO,1634946989,MARHABE HOTEL
70,70,Dingle,10.9995,122.6711,80.24,85,81,9.93,PH,1634946990,Could not find result
78,78,Makkah al Mukarramah,21.4267,39.8261,82.00,40,24,2.10,SA,1634946742,"Makkah Clock Royal Tower, A Fairmont Hotel"
95,95,Kupang,-10.1667,123.5833,84.78,74,75,8.05,ID,1634946996,ASTON Kupang Hotel & Convention Center
108,108,Kapaa,22.0752,-159.3190,82.38,77,19,5.01,US,1634946998,The ISO
110,110,Coahuayana Viejo,18.7333,-103.6833,87.28,86,92,4.25,MX,1634946952,Hotel Los Arcos
112,112,Muncar,-8.4333,114.3333,80.51,78,100,3.94,ID,1634946998,Could not find result
119,119,Kavaratti,10.5669,72.6420,81.23,77,58,4.03,IN,1634946743,Hotel Al-Be & Restaurant


In [53]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [56]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))